In [ ]:
#Create and connect Mysql DataBase
import mysql.connector
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
)
mycursor = mydb.cursor(buffered=True)
mycursor.execute("CREATE DATABASE RED_BUS_PROJECT") #creating database

mycursor.execute("use RED_BUS_PROJECT")
#create table inside the database
mycursor.execute('''CREATE TABLE Bus_Details(ID INT AUTO_INCREMENT PRIMARY KEY,
Route_Name TEXT NULL,
Route_Link TEXT NULL,
Bus_Name VARCHAR(255) NOT NULL,
Bus_Type VARCHAR(255) NOT NULL,
Departing_Time TIME NOT NULL,
Duration VARCHAR(255) NOT NULL,
Reaching_Time TIME NOT NULL,
Star_Rating FLOAT NULL,
Price FLOAT NULL,
Seats_available INT(8) NULL
)''')

In [ ]:
#function to reconnect in case of any connection issue
def reconnect():
    global mydb, mycursor
    mydb = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "",
        database = "red_bus_project"
    )
    mycursor = mydb.cursor()

In [ ]:
#Inserting datas to table, scraped from redbus website
insert_data = '''INSERT INTO bus_details(
Route_Name,
Route_Link,
Bus_Name,
Bus_Type,
Departing_Time,
Duration,
Reaching_Time,
Star_Rating,
Price,
Seats_available)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

data = final_df.values.tolist()
batch_size = 1000  # Adjust this value as needed

for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
        # Performing batch insert with executemany()
        mycursor.executemany(insert_data, batch_data)
        mydb.commit()
    except mysql.connector.Error as err:
        if err.errno == mysql.connector.errorcode.CR_SERVER_LOST or err.errno == mysql.connector.errorcode.CR_SERVER_GONE_ERROR:
            reconnect()
            mycursor.executemany(insert_data, batch_data)
            mydb.commit()